In [1]:
using Distributions, StatsBase, TimeIt, ProgressMeter
using PyPlot
import PyPlot
include("GenInvGaussian.jl")
include("gibbs_zigzag.jl")

# Define abstract type for gaussian prior, sub-types of this abstract types must have attributes mu and sigma2  
abstract type prior end
abstract type gaussian_prior <:prior end
abstract type laplace_prior <:prior end

In [21]:
function Gibbs_zigzag(our_prior::DL_prior, y::Array{Float64}, ξ0::Array{Float64}, n_iter::Int64) 
    n = length(y)
    samples_param = zeros(n_iter, n)
    samples_hyperparam = zeros(n_iter+1, 2n+1)
    ξ = copy(ξ0)
    θ = ones(n)
    @showprogress for iter in 1:n_iter 
        σ2 = get_σ2(our_prior)
        ξ, θ, _1, _2 = block_Gibbs_update_params(our_prior, y, ξ, θ, ones(n)*1e-6)
        our_prior = block_Gibbs_update_hypparams(our_prior, ξ)
        samples_param[iter,:] = copy(ξ)
        samples_hyperparam[iter,1:n] = our_prior.ψ
        samples_hyperparam[iter,n+1:2n] = our_prior.ϕ
        samples_hyperparam[iter,end] = our_prior.τ
    end
    return samples_param, samples_hyperparam
end     

Gibbs_zigzag (generic function with 1 method)

In [22]:
n = 100
y = rand(Binomial(),n) + 0.
our_prior = DL_prior(n, rand(n), rand(n), rand(), 0.5);

In [6]:
our_prior.a

0.5

In [23]:
ξ0 = rand(n) 
n_iter = 10^2
samples_param = zeros(n_iter, n)
samples_hyperparam = zeros(n_iter+1, 2n+1)
ξ = copy(ξ0)
θ = ones(n);

In [24]:
σ2 = get_σ2(our_prior)
ξ, θ, _1, _2 = block_Gibbs_update_params(our_prior, y, ξ, θ, ones(n)*1e-6)

([0.238377, 0.685405, 0.341406, 0.355973, 0.805604, 0.665487, 0.224805, 0.990808, 0.980752, 0.653223  …  0.304237, 0.0689562, 0.711705, 0.910078, 0.948613, 0.916614, 0.857441, 0.480932, 0.205787, 0.794139], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])

In [28]:
β = copy(ξ);

In [29]:
@timeit our_prior.ψ = [rand(InverseGaussian(our_prior.ϕ[j]*our_prior.τ/abs.(β[j]), 1)) for j in 1:our_prior.n]
@timeit our_prior.τ = rand_GenInvGaussian(1-our_prior.n, 1, 2sum(abs.(β)./our_prior.ϕ))
@timeit T = [rand_GenInvGaussian(our_prior.a-1, 1, 2abs(β[j])) for j in 1:our_prior.n]
@timeit our_prior.ϕ = T/sum(T)

10000 loops, best of 3: 38.88 µs per loop
100000 loops, best of 3: 7.99 µs per loop


LoadError: [91mInterruptException:[39m

In [25]:
our_prior = block_Gibbs_update_hypparams(our_prior, ξ)

LoadError: [91mInterruptException:[39m

In [37]:
ξ0 = rand(n)
n_iter = 10^2
samples_param, samples_hyperparam = Gibbs_zigzag(our_prior, y, ξ0, n_iter) 

LoadError: [91mInterruptException:[39m

In [38]:
n = length(y)
samples_param = zeros(n_iter, n)
samples_hyperparam = zeros(n_iter+1, 2n+1)
ξ = copy(ξ0)
θ = ones(n);

In [39]:
@showprogress for iter in 1:n_iter 
    σ2 = get_σ2(our_prior)
    ξ, θ, _, __ = block_Gibbs_update_params(our_prior, y, ξ, θ, ones(n)*1e-6)
    our_prior = block_Gibbs_update_hypparams(our_prior, ξ)
    samples_param[iter,:] = copy(ξ)
    samples_hyperparam[iter,1:n] = our_prior.ψ
    samples_hyperparam[iter,n+1:2n] = our_prior.ϕ
    samples_hyperparam[iter,end] = our_prior.τ
end

LoadError: [91mInterruptException:[39m

In [56]:
σ2 = get_σ2(our_prior)
ξ, θ, _1, _2 = block_Gibbs_update_params(our_prior, y, ξ, θ, σ2)
# our_prior = block_Gibbs_update_hypparams(our_prior, ξ)
# samples_param[iter,:] = copy(ξ)
# samples_hyperparam[iter,1:n] = our_prior.ψ
# samples_hyperparam[iter,n+1:2n] = our_prior.ϕ
# samples_hyperparam[iter,end] = our_prior.τ

([-3.25192, 34.0855, 9.28155, -5.21782, -14.1643, -39.6045, 0.251903, 0.443602, 1.51574, 18.3966  …  0.783151, 1.17662, 0.764553, 0.87581, 0.266763, 0.193569, 0.613415, 0.364144, 0.534146, -0.067337], [1.0, -1.0, -1.0, 1.0, 1.0, 1.0, 1.0, -1.0, -1.0, -1.0  …  -1.0, 1.0, 1.0, -1.0, -1.0, -1.0, -1.0, 1.0, -1.0, -1.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0], [3.0, 49.0, 11.0, 7.0, 15.0, 38.0, 2.0, 0.0, 0.0, 15.0  …  2.0, 0.0, 0.0, 1.0, 0.0, 1.0, 2.0, 0.0, 0.0, 1.0])

In [57]:
our_prior = block_Gibbs_update_hypparams(our_prior, ξ)

LoadError: [91mInterruptException:[39m

In [ ]:
our_prior.ψ = [rand(InverseGaussian(our_prior.ϕ[j]*our_prior.τ/abs.(ξ[j]), 1)) for j in 1:our_prior.n]
our_prior.τ = rand_GenInvGaussian(1-our_prior.n, 1, 2sum(abs.(ξ)./our_prior.ϕ))
T = [rand_GenInvGaussian(our_prior.a-1, 1, 2abs(ξ[j])) for j in 1:our_prior.n]
our_prior.ϕ = T/sum(T)

In [62]:
T = [rand_GenInvGaussian(our_prior.a-1, 1, 2abs(ξ[j])) for j in 1:our_prior.n]

LoadError: [91mInterruptException:[39m

In [65]:
@timeit [rand_GenInvGaussian(1-our_prior.n, 1, 2sum(abs.(ξ)./our_prior.ϕ)) for i in 1:our_prior.n];

100 loops, best of 3: 3.12 ms per loop


In [74]:
@timeit [rand_GenInvGaussian(-20000, 1, 2abs(ξ[j])) for j in 1:1];

100000 loops, best of 3: 1.29 µs per loop


In [67]:
our_prior.a-1, 1, 2abs(ξ[1])

(-0.5, 1, 6.503839341405488)

compute_configT (generic function with 1 method)